In [6]:
from sportsipy.nfl.boxscore import Boxscore, BoxscorePlayer, Boxscores
from sportsipy.nfl.roster import Roster, Player, AbstractPlayer
from sportsipy.nfl.schedule import Game, Schedule
from sportsipy.nfl.teams import Team, Teams
from sportsipy.nfl.constants import *

import pandas as pd
pd.set_option('display.max_columns', None)

from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# df = pd.read_pickle('data/df_boxscores.pkl')

In [57]:
import pickle5 as pickle
with open('data/df_boxscores.pkl', "rb") as fh:
  team_data = pickle.load(fh)

# String of full day of the week (Sunday, Monday, Thursday)
team_data['day_of_week'] = team_data['date'].str.split(expand=True)[0]
# Abbreviation of month as a string
team_data['month'] = team_data['date'].str.split(expand=True)[1]
# Temperature as integer in Farenheight
team_data['temperature'] = team_data['weather'].str.split(',', expand=True)[0].str.split(expand=True)[0]
# Relative humidity as a percentage
team_data['humidity'] = team_data['weather'].str.split(',', expand=True)[1].str.split(expand=True)[2]
# Wind chill as integer
df_boxscore['wind_chill'] = df_boxscore['weather'].str.split(',', expand=True)[3].str.split(expand=True)[2]

# Wind in MPH
df_boxscore['wind'] = df_boxscore['weather'].str.split(',', expand=True)[2]
# Converts any games played in a dome to 0, originally listed as NoneType
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if x is None else x)
# Converts any games listed as 'no wind' to 0
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if str(x)[-1:] == 'd' else x)
# Changing all values to the integers
team_data['wind'] = team_data['wind'].str.split(expand=True)[1].fillna(0).astype('int64')

team_data.head()

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_or_under,over_under,over_under_total,roof,stadium,surface,time,total_minus_over,total_points,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
200009030min,64104,23,0,0,2,1,0,272,41,27,2,290,5,27,25,1,153,15,9,32:34,2,425,1,45,18,"Sunday Sep 3, 2000",2000-09-03 13:00:00,3:08,17,0,0,2,0,1,188,23,13,0,190,4,30,30,3,186,13,6,27:26,1,374,1,20,2,CHI,Chicago Bears,over,46.5 (over),46.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,1:00pm,10.5,57,Minnesota Vikings -4.5,None,Home,MIN,Minnesota Vikings,Vikings
200009030kan,78357,20,0,0,1,1,1,267,32,22,1,273,7,27,32,1,119,13,6,33:13,1,386,2,78,6,"Sunday Sep 3, 2000",2000-09-03 13:00:00,2:58,17,1,0,0,0,1,206,37,16,2,212,6,14,24,0,74,15,5,26:47,1,280,1,50,6,KAN,Kansas City Chiefs,under,44.0 (under),44.0,Outdoors,Arrowhead Stadium,Grass,1:00pm,-3.0,41,Indianapolis Colts -3.5,"90 degrees, relative humidity 50%, wind 5 mph,...",Away,CLT,Indianapolis Colts,Chiefs
200009030was,80257,12,1,1,1,1,0,124,26,17,1,183,9,17,20,0,112,11,2,26:46,6,236,1,65,59,"Sunday Sep 3, 2000",2000-09-03 13:01:00,3:04,23,1,0,0,0,0,234,36,25,0,234,7,20,31,2,162,12,5,33:14,0,396,0,64,0,CAR,Carolina Panthers,under,47.5 (under),47.5,Outdoors,FedExField,Grass,1:01pm,-10.5,37,Washington Redskins -10.5,"80 degrees, relative humidity 98%, no wind, wi...",Home,WAS,Washington Redskins,Redskins
200009030cle,72418,28,0,0,1,0,0,279,34,24,1,301,1,27,40,2,119,16,10,37:03,4,398,0,5,22,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:46,9,2,1,1,1,0,153,27,19,1,160,3,7,16,0,96,10,3,22:57,1,249,1,25,7,CLE,Cleveland Browns,under,38.5 (under),38.5,Outdoors,Cleveland Browns Stadium,Grass,1:02pm,-4.5,34,Jacksonville Jaguars -10.5,"78 degrees, relative humidity 63%, wind 6 mph,...",Away,JAX,Jacksonville Jaguars,Jaguars
200009030atl,54626,23,1,0,3,0,1,247,36,23,3,253,7,28,24,1,92,9,2,28:21,1,339,1,42,6,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:13,22,0,0,2,1,0,264,31,16,2,264,4,36,32,0,95,11,4,31:39,0,359,1,51,0,SFO,San Francisco 49ers,over,46.5 (over),46.5,Dome,Georgia Dome,Astroturf,1:02pm,17.5,64,Atlanta Falcons -6.5,None,Home,ATL,Atlanta Falcons,49ers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202101170kan,16730,21,3,3,1,1,1,196,37,23,1,204,3,17,22,1,112,13,6,29:23,1,308,2,25,8,"Sunday Jan 17, 2021",2021-01-17 15:05:00,3:01,24,2,2,0,0,1,315,38,27,1,321,7,22,24,1,123,10,5,30:37,1,438,1,55,6,CLE,Cleveland Browns,under,55.5 (under),55.5,Outdoors,Arrowhead Stadium,Astroturf,3:05pm,-16.5,39,Kansas City Chiefs -7.5,"40 degrees, relative humidity 62%, wind 13 mph",Home,KAN,Kansas City Chiefs,Browns (deferred)
202101170nor,3750,20,1,1,1,0,0,189,33,18,2,199,7,30,35,1,127,17,8,31:31,1,316,0,62,10,"Sunday Jan 17, 2021",2021-01-17 18:40:00,3:12,20,0,0,1,1,3,190,35,20,2,190,5,20,25,0,104,11,6,28:29,0,294,

## Current/Ongoing situations

### Seems like anything not played outdoors has nonetypes for the following columns:

```python
'temperature', 'humidity', 'wind_chill'
```

- Accounts for 1,404 individual games, appears to leave some other NaN values, unknown as to why for now

In [104]:
team_data['roof'].unique()

array(['Dome', 'Outdoors', 'Retractable Roof (Closed)',
       'Retractable Roof (Open)'], dtype=object)

In [102]:
team_data[team_data['roof'].isin(['Dome', 'Retractable Roof (Open)',
                                  'Retractable Roof (Closed)'])]#['wind_chill'].unique() #'temperature', 'humidity', 
                                                                  #'wind_chill']]#.fillna(0, inplace=True)

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_or_under,over_under,over_under_total,roof,stadium,surface,time,total_minus_over,total_points,vegas_line,weather,winner,winning_abbr,winning_name,won_toss,day_of_week,month,temperature,humidity,wind,wind_chill
200009030min,64104,23,0,0,2,1,0,272,41,27,2,290,5,27,25,1,153,15,9,32:34,2,425,1,45,18,"Sunday Sep 3, 2000",2000-09-03 13:00:00,3:08,17,0,0,2,0,1,188,23,13,0,190,4,30,30,3,186,13,6,27:26,1,374,1,20,2,CHI,Chicago Bears,over,46.5 (over),46.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,1:00pm,10.5,57,Minnesota Vikings -4.5,None,Home,MIN,Minnesota Vikings,Vikings,Sunday,Sep,None,None,0,None
200009030atl,54626,23,1,0,3,0,1,247,36,23,3,253,7,28,24,1,92,9,2,28:21,1,339,1,42,6,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:13,22,0,0,2,1,0,264,31,16,2,264,4,36,32,0,95,11,4,31:39,0,359,1,51,0,SFO,San Francisco 49ers,over,46.5 (over),46.5,Dome,Georgia Dome,Astroturf,1:02pm,17.5,64,Atlanta Falcons -6.5,None,Home,ATL,Atlanta Falcons,49ers,Sunday,Sep,None,None,0,None
200009030nor,64900,10,1,0,2,1,1,89,25,13,0,100,8,14,32,0,98,16,4,30:49,2,187,2,74,11,"Sunday Sep 3, 2000",2000-09-03 13:03:00,3:12,13,1,0,5,2,1,146,34,18,0,169,12,10,25,0,106,17,6,29:11,5,252,3,89,23,NOR,New Orleans Saints,under,40.5 (under),40.5,Dome,Louisiana Superdome,Astroturf,1:03pm,-16.5,24,Pick,None,Away,DET,Detroit Lions,Lions,Sunday,Sep,None,None,0,None
200009040ram,65956,25,1,0,1,0,0,274,29,19,2,307,5,36,28,1,150,11,5,32:13,4,424,0,27,33,"Monday Sep 4, 2000",2000-09-04 21:07:00,3:00,23,0,0,0,0,3,433,35,25,3,441,5,41,18,2,80,7,6,27:47,2,513,3,56,8,DEN,Denver Broncos,over,48.5 (over),48.5,Dome,TWA Dome,Astroturf,9:07pm,28.5,77,St. Louis Rams -6.5,None,Home,RAM,St. Louis Rams,Rams,Monday,Sep,None,None,0,None
200009100min,64112,10,0,0,0,0,1,160,31,12,1,175,7,7,16,0,49,13,3,22:41,3,209,1,40,15,"Sunday Sep 10, 2000",2000-09-10 12:00:00,3:04,24,0,0,3,1,3,343,37,23,1,355,7,13,28,0,125,11,5,37:19,3,468,4,60,12,MIA,Miami Dolphins,under,42.5 (under),42.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,12:00pm,-22.5,20,Minnesota Vikings -3.0,None,Home,MIN,Minnesota Vikings,Vikings,Sunday,Sep,None,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202101030ram,0,15,0,0,1,0,1,166,27,19,1,192,6,7,18,0,48,9,2,22:09,4,214,1,38,26,"Sunday Jan 3, 2021",2021-01-03 16:25:00,2:52,21,1,1,1,1,1,223,38,22,0,231,5,18,30,0,110,15,6,37:51,2,333,2,38,8,CRD,Arizona Cardinals,under,42.0 (under),42.0,Dome,SoFi Stadium,Matrixturf,4:25pm,-17.0,25,Los Angeles Rams -1.0,None,Home,RAM,Los Angeles Rams,Rams (deferred),Sunday,Jan,None,None,0,None
202101030htx,12504,28,1,0,2,1,0,204,27,18,1,216,6,41,41,4,288,10,7,31:04,2,492,1,50,12,"Sunday Jan 3, 2021",2021-01-03 16:25:00,3:13,26,1,1,0,0,1,361,40,29,3,385,6,38,17,1,96,12,5,28:56,4,457,1,44

In [96]:
team_data[team_data['roof'] == 'Dome']

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_or_under,over_under,over_under_total,roof,stadium,surface,time,total_minus_over,total_points,vegas_line,weather,winner,winning_abbr,winning_name,won_toss,day_of_week,month,temperature,humidity,wind,wind_chill
200009030min,64104,23,0,0,2,1,0,272,41,27,2,290,5,27,25,1,153,15,9,32:34,2,425,1,45,18,"Sunday Sep 3, 2000",2000-09-03 13:00:00,3:08,17,0,0,2,0,1,188,23,13,0,190,4,30,30,3,186,13,6,27:26,1,374,1,20,2,CHI,Chicago Bears,over,46.5 (over),46.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,1:00pm,10.5,57,Minnesota Vikings -4.5,None,Home,MIN,Minnesota Vikings,Vikings,Sunday,Sep,None,None,0,None
200009030atl,54626,23,1,0,3,0,1,247,36,23,3,253,7,28,24,1,92,9,2,28:21,1,339,1,42,6,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:13,22,0,0,2,1,0,264,31,16,2,264,4,36,32,0,95,11,4,31:39,0,359,1,51,0,SFO,San Francisco 49ers,over,46.5 (over),46.5,Dome,Georgia Dome,Astroturf,1:02pm,17.5,64,Atlanta Falcons -6.5,None,Home,ATL,Atlanta Falcons,49ers,Sunday,Sep,None,None,0,None
200009030nor,64900,10,1,0,2,1,1,89,25,13,0,100,8,14,32,0,98,16,4,30:49,2,187,2,74,11,"Sunday Sep 3, 2000",2000-09-03 13:03:00,3:12,13,1,0,5,2,1,146,34,18,0,169,12,10,25,0,106,17,6,29:11,5,252,3,89,23,NOR,New Orleans Saints,under,40.5 (under),40.5,Dome,Louisiana Superdome,Astroturf,1:03pm,-16.5,24,Pick,None,Away,DET,Detroit Lions,Lions,Sunday,Sep,None,None,0,None
200009040ram,65956,25,1,0,1,0,0,274,29,19,2,307,5,36,28,1,150,11,5,32:13,4,424,0,27,33,"Monday Sep 4, 2000",2000-09-04 21:07:00,3:00,23,0,0,0,0,3,433,35,25,3,441,5,41,18,2,80,7,6,27:47,2,513,3,56,8,DEN,Denver Broncos,over,48.5 (over),48.5,Dome,TWA Dome,Astroturf,9:07pm,28.5,77,St. Louis Rams -6.5,None,Home,RAM,St. Louis Rams,Rams,Monday,Sep,None,None,0,None
200009100min,64112,10,0,0,0,0,1,160,31,12,1,175,7,7,16,0,49,13,3,22:41,3,209,1,40,15,"Sunday Sep 10, 2000",2000-09-10 12:00:00,3:04,24,0,0,3,1,3,343,37,23,1,355,7,13,28,0,125,11,5,37:19,3,468,4,60,12,MIA,Miami Dolphins,under,42.5 (under),42.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,12:00pm,-22.5,20,Minnesota Vikings -3.0,None,Home,MIN,Minnesota Vikings,Vikings,Sunday,Sep,None,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202012270sdg,0,23,2,2,0,0,2,263,47,24,0,264,6,16,28,1,133,17,7,33:09,2,396,2,44,1,"Sunday Dec 27, 2020",2020-12-27 16:05:00,3:00,16,1,1,1,0,0,227,33,21,1,253,6,19,20,0,89,12,3,26:51,2,316,0,45,26,DEN,Denver Broncos,under,47.5 (under),47.5,Dome,SoFi Stadium,Matrixturf,4:05pm,-12.5,35,Los Angeles Chargers -2.5,None,Home,SDG,Los Angeles Chargers,Broncos (deferred),Sunday,Dec,None,None,0,None
202101030det,0,31,2,2,2,0,0,383,40,28,3,405,4,37,31,2,125,13,4,35:13,3,508,0,35,22,"Sunday Jan 3, 2021",2021-01-03 13:00:00,3:07,21,1,1,1,1,1,288,31,20,3,293,7,35,22,2,129,10,5,24:47,1,

### Looking for a way to indicate whether a game was played in Prime Time (Basically after 7 PM)

In [76]:
# Could be used to get the time of day from time column
dt = pd.to_datetime(team_data['time'], format='%I:%M%p').dt.time

### Not able to find the duration for this anywhere at the moment

- This is the only game we run into this situation, may want to just remove it

In [80]:
# Not able to find the duration for this anywhere at the moment
team_data[team_data['duration'].isna() == True]

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_or_under,over_under,over_under_total,roof,stadium,surface,time,total_minus_over,total_points,vegas_line,weather,winner,winning_abbr,winning_name,won_toss,day_of_week,month,temperature,humidity,wind,wind_chill
200209220oti,68804,26,0,0,4,3,1,311,50,36,3,326,7,31,20,0,92,15,8,31:02,3,403,4,57,15,"Sunday Sep 22, 2002",2002-09-22 13:03:00,None,13,1,1,1,0,0,101,23,15,2,107,13,28,34,1,86,13,3,33:11,1,187,0,96,6,OTI,Tennessee Titans,over,40.5 (over),40.5,Outdoors,The Coliseum,Grass,1:03pm,18.5,59,Tennessee Titans -4.5,"72 degrees, relative humidity 78%, wind 9 mph,...",Away,CLE,Cleveland Browns,Browns,Sunday,Sep,72,78%,9,0


In [78]:
team_data[['attendance', 'date', 'datetime', 'day_of_week', 'duration', 'humidity', 'losing_abbr', 
           'losing_name', 'month', 'over_or_under', 'over_under', 'over_under_total', 'roof', 
           'stadium', 'surface', 'time', 'temperature', 'total_minus_over', 'total_points', 
           'vegas_line', 'weather', 'wind','wind_chill', 'winner',
       'winning_abbr', 'winning_name', 'won_toss']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 5593 entries, 200009030min to 202102070tam
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   attendance        5593 non-null   int64         
 1   date              5593 non-null   object        
 2   datetime          5593 non-null   datetime64[ns]
 3   day_of_week       5593 non-null   object        
 4   duration          5592 non-null   object        
 5   humidity          4149 non-null   object        
 6   losing_abbr       5593 non-null   object        
 7   losing_name       5593 non-null   object        
 8   month             5593 non-null   object        
 9   over_or_under     5593 non-null   object        
 10  over_under        5593 non-null   object        
 11  over_under_total  5593 non-null   float64       
 12  roof              5593 non-null   object        
 13  stadium           5593 non-null   object        
 14  surface   

# Steps to setup `get_data.py` file to get all information into DataFrame

1) Loop through Boxscores(`week=1`, `year={looped_number}`, `end_week=22`)
    - Returns Dictionary that will contain all game boxscores as strings like: `202009130min`
    - Get all these strings into a list to also loop through

[Sportsipy docs](https://sportsreference.readthedocs.io/en/stable/nfl.html#module-sportsipy.nfl.boxscore)

[Sportsipy Github](https://github.com/roclark/sportsipy/tree/ea0043747015209550abeee15df75914a58fe40b/sportsipy/nfl)

# Boxscore

* Boxscore(`Game String`)

## What was added to the created DataFrames from the API

```python
# String of full day of the week (Sunday, Monday, Thursday)
team_data['day_of_week'] = team_data['date'].str.split(expand=True)[0]
# Abbreviation of month as a string
team_data['month'] = team_data['date'].str.split(expand=True)[1]
# Temperature as integer in Farenheight
team_data['temperature'] = team_data['weather'].str.split(',', expand=True)[0].str.split(expand=True)[0]
# Relative humidity as a percentage
team_data['humidity'] = team_data['weather'].str.split(',', expand=True)[1].str.split(expand=True)[2]
# Wind chill as integer
df_boxscore['wind_chill'] = df_boxscore['weather'].str.split(',', expand=True)[3].str.split(expand=True)[2]

# Wind in MPH
df_boxscore['wind'] = df_boxscore['weather'].str.split(',', expand=True)[2]
# Converts any games played in a dome to 0, originally listed as NoneType
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if x is None else x)
# Converts any games listed as 'no wind' to 0
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if str(x)[-1:] == 'd' else x)
# Changing all values to the integers
team_data['wind'] = team_data['wind'].str.split(expand=True)[1].fillna(0).astype('int64')
```

In [11]:
boxscore = Boxscore('202009130min')
df_box = boxscore.dataframe

In [36]:
game_uris = []

for idx, week in enumerate(boxscores.keys()):
    for score in boxscores[week]:
        game_uris.append(score['boxscore'])
        
game_uris

['202009100kan',
 '202009130buf',
 '202009130atl',
 '202009130was',
 '202009130rav',
 '202009130nwe',
 '202009130min',
 '202009130jax',
 '202009130det',
 '202009130car',
 '202009130cin',
 '202009130sfo',
 '202009130nor',
 '202009130ram',
 '202009140nyg',
 '202009140den',
 '202009170cle',
 '202009200clt',
 '202009200chi',
 '202009200tam',
 '202009200pit',
 '202009200phi',
 '202009200oti',
 '202009200nyj',
 '202009200mia',
 '202009200gnb',
 '202009200dal',
 '202009200crd',
 '202009200sdg',
 '202009200htx',
 '202009200sea',
 '202009210rai',
 '202009240jax',
 '202009270buf',
 '202009270atl',
 '202009270pit',
 '202009270phi',
 '202009270nyg',
 '202009270nwe',
 '202009270min',
 '202009270cle',
 '202009270sdg',
 '202009270clt',
 '202009270sea',
 '202009270den',
 '202009270crd',
 '202009270nor',
 '202009280rav',
 '202010010nyj',
 '202010040chi',
 '202010040car',
 '202010040was',
 '202010040tam',
 '202010040mia',
 '202010040htx',
 '202010040det',
 '202010040dal',
 '202010040cin',
 '202010040ram

In [63]:
game_df = pd.concat([final_df, second_df], axis=0)
game_df

completed_passes  \
year game_uri     away_abbreviation name                                
2000 200009030min MIN               Daunte Culpepper               13   
                                    Robert Smith                    0   
                                    Jim Kleinsasser                 0   
                                    Randy Moss                      0   
                                    John Davis                      0   
...                                                               ...   
2021 202102070tam TAM               Bradley Pinion               None   
                                    Vita Vea                     None   
                                    William Gholston             None   
                                    Jaydon Mickens               None   
                                    Ryan Succop                  None   

                                                     attempted_passes  \
year game_uri     away_abbreviation name                                
2000 200009030min MIN               Daunte Culpepper               23   
                                    Robert Smith                    0   
                                    Jim Kleinsasser                 0   
                                    Randy Moss                      0   
                                    John Davis                      0   
...                                                               ...   
2021 202102070tam TAM               Bradley Pinion               None   
                                    Vita Vea                     None   
                                    William Gholston             None   
                                    Jaydon Mickens               None   
                                    Ryan Succop                  None   

                                                     passing_yards  \
year game_uri     away_abbreviation name                             
2000 200009030min MIN               Daunte Culpepper           190   
                                    Robert Smith                 0   
                                    Jim Kleinsasser              0   
                                    Randy Moss                   0   
                                    John Davis                   0   
...                                                            ...   
2021 202102070tam TAM               Bradley Pinion            None   
                                    Vita Vea                  None   
                                    William Gholston          None   
                                    Jaydon Mickens            None   
                                    Ryan Succop               None   

                                                     passing_touchdowns  \
year game_uri     away_abbreviation name                                  
2000 200009030min MIN               Daunte Culpepper                  0   
                                    Robert Smith                      0   
                                    Jim Kleinsasser                   0   
                                    Randy Moss                        0   
                                    John Davis                        0   
...                                                                 ...   
2021 202102070tam TAM               Bradley Pinion                 None   
                                    Vita Vea                       None   
                                    William Gholston               None   
                                    Jaydon Mickens                 None   
                                    Ryan Succop                    None   

                                                     interceptions_thrown  \
year game_uri     away_abbreviation name                                    
2000 200009030min MIN               Daunte Culpepper                    1   
                                   

# Roster

In [2]:
roster = Roster(team, year)
roster.coach, roster.players

('Matt LaFleur',
 [Davante Adams (AdamDa01),
  Montravius Adams (AdamMo00),
  Jaire Alexander (AlexJa00),
  Adrian Amos (AmosAd00),
  Tavon Austin (AustTa00),
  David Bakhtiari (BakhDa00),
  Krys Barnes (BarnKr00),
  Reggie Begelton (BegeRe00),
  Henry Black (BlacHe00),
  Tim Boyle (BoylTi00),
  Ben Braden (BradBe00),
  Hunter Bradley (BradHu00),
  James Burgess (BurgJa01),
  Oren Burks (BurkOr00),
  Kenny Clark (ClarKe01),
  Mason Crosby (CrosMa20),
  Dominique Dafney (DafnDo00),
  Josiah Deguara (DeguJo00),
  AJ Dillon (DillAJ00),
  Tyler Ervin (ErviTy00),
  Tipa Galeai (GaleTi01),
  Jonathan Garvin (GarvJo00),
  Rashan Gary (GaryRa00),
  Raven Greene (GreeRa02),
  De'Jon Harris (HarrDe10),
  Damon Harrison (HarrDa03),
  Ka'dar Hollman (HollKa00),
  Josh Jackson (JackJo01),
  Elgton Jenkins (JenkEl00),
  Aaron Jones (JoneAa00),
  Kingsley Keke (KekeKi00),
  Kevin King (KingKe01),
  Christian Kirksey (KirkCh00),
  Tyler Lancaster (LancTy00),
  Allen Lazard (LazaAl00),
  Marcedes Lewis

In [4]:
player = Player(player_id)
player

Davante Adams (AdamDa01)

In [5]:
player.dataframe

,adjusted_net_yards_per_attempt_index,adjusted_net_yards_per_pass_attempt,adjusted_yards_per_attempt,adjusted_yards_per_attempt_index,all_purpose_yards,approximate_value,assists_on_tackles,attempted_passes,birth_date,blocked_punts,catch_percentage,completed_passes,completion_percentage_index,drop_percentage,dropped_passes,espn_qbr,extra_point_percentage,extra_points_attempted,extra_points_made,field_goal_percentage,field_goals_attempted,field_goals_made,fifty_plus_yard_field_goal_attempts,fifty_plus_yard_field_goals_made,first_downs_receiving,first_downs_rushing,fourth_quarter_comebacks,fourty_to_fourty_nine_yard_field_goal_attempts,fourty_to_fourty_nine_yard_field_goals_made,fumbles,fumbles_forced,fumbles_recovered,fumbles_recovered_for_touchdown,game_winning_drives,games,games_started,height,interception_percentage,interception_percentage_index,interceptions,interceptions_returned_for_touchdown,interceptions_thrown,kickoff_return_touchdown,kickoff_return_yards,kickoff_returns,less_than_nineteen_yards_field_goal_attempts,less_than_nineteen_yards_field_goals_made,longest_field_goal_made,longest_interception_return,longest_kickoff_return,longest_pass,longest_punt,longest_punt_return,longest_reception,longest_rush,name,net_yards_per_attempt_index,net_yards_per_pass_attempt,passer_rating_index,passes_defended,passing_completion,passing_touchdown_percentage,passing_touchdowns,passing_yards,passing_yards_per_attempt,player_id,position,punt_return_touchdown,punt_return_yards,punt_returns,punts,qb_record,quarterback_rating,receiving_broken_tackles,receiving_touchdowns,receiving_yards,receiving_yards_after_catch,receiving_yards_after_catch_per_reception,receiving_yards_before_catch,receiving_yards_before_catch_per_reception,receiving_yards_per_game,receiving_yards_per_reception,receptions,receptions_per_broken_tackle,receptions_per_game,rush_attempts,rush_attempts_per_broken_tackle,rush_attempts_per_game,rush_broken_tackles,rush_touchdowns,rush_yards,rush_yards_after_contact,rush_yards_after_contact_per_attempt,rush_yards_before_contact,rush_yards_before_contact_per_attempt,rush_yards_per_attempt,rush_yards_per_game,rushing_and_receiving_touchdowns,sack_percentage,sack_percentage_index,sacks,safeties,season,tackles,team_abbreviation,thirty_to_thirty_nine_yard_field_goal_attempts,thirty_to_thirty_nine_yard_field_goals_made,times_pass_target,times_sacked,total_punt_yards,touchdown_percentage_index,touches,twenty_to_twenty_nine_yard_field_goal_attempts,twenty_to_twenty_nine_yard_field_goals_made,weight,yards_from_scrimmage,yards_lost_to_sacks,yards_per_attempt_index,yards_per_completed_pass,yards_per_game_played,yards_per_kickoff_return,yards_per_punt,yards_per_punt_return,yards_per_touch,yards_recovered_from_fumble,yards_returned_from_interception
2014,None,None,None,None,None,6,None,None,1992-12-24,None,57.6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,None,None,None,None,16,11,6-1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,45,None,Davante Adams,None,None,None,None,None,None,None,None,None,AdamDa01,WR,None,None,None,None,None,None,None,3,446,None,None,None,None,27.9,11.7,38,None,2.4,None,None,None,None,None,None,None,None,None,None,None,None,3,None,None,None,None,2014,None,GNB,None,None,66,None,None,None,38,None,None,215,446,None,None,None,None,None,None,None,11.7,None,None
2015,None,None,None,None,None,4,None,None,1992-12-24,None,53.2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,None,None,None,None,13,12,6-1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,40,None,Davante Adams,None,None,None,None,None,None,None,None,None,AdamDa01,WR,None,None,None,None,None,None,None,1,483,None,None,None,None,37.2,9.7,50,None,3.8,None,None,None,None,None,None,None,None,None,None,None,None,1,None,None,None,None,2015,None,GNB,None,None,94,None,None,None,50,None,None,215,483,None,None,None,None

# Schedule

## Need to get schedules for this year, looks like it is available now

In [106]:
schedule = Schedule('GNB', 2021)
schedule

September 12 - NOR
September 20 - DET
September 26 - SFO
October 3 - PIT
October 10 - CIN
October 17 - CHI
October 24 - WAS
October 28 - CRD
November 7 - KAN
November 14 - SEA
November 21 - MIN
November 28 - RAM
December 12 - CHI
December 19 - RAV
December 25 - CLE
January 2 - MIN
January 9 - DET

In [110]:
schedule.dataframe#[17]

,boxscore_index,date,datetime,day,extra_points_attempted,extra_points_made,field_goals_attempted,field_goals_made,fourth_down_attempts,fourth_down_conversions,interceptions,location,opponent_abbr,opponent_name,overtime,pass_attempts,pass_completion_rate,pass_completions,pass_touchdowns,pass_yards,pass_yards_per_attempt,points_allowed,points_scored,punt_yards,punts,quarterback_rating,result,rush_attempts,rush_touchdowns,rush_yards,rush_yards_per_attempt,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,type,week,yards_lost_from_sacks
202109120nor,202109120nor,September 12,2021-09-12,Sun,None,None,None,None,None,None,None,Away,NOR,New Orleans Saints,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,1,None
202109200gnb,202109200gnb,September 20,2021-09-20,Mon,None,None,None,None,None,None,None,Home,DET,Detroit Lions,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,2,None
202109260sfo,202109260sfo,September 26,2021-09-26,Sun,None,None,None,None,None,None,None,Away,SFO,San Francisco 49ers,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,3,None
202110030gnb,202110030gnb,October 3,2021-10-03,Sun,None,None,None,None,None,None,None,Home,PIT,Pittsburgh Steelers,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,4,None
202110100cin,202110100cin,October 10,2021-10-10,Sun,None,None,None,None,None,None,None,Away,CIN,Cincinnati Bengals,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,5,None
202110170chi,202110170chi,October 17,2021-10-17,Sun,None,None,None,None,None,None,None,Away,CHI,Chicago Bears,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,6,None
202110240gnb,202110240gnb,October 24,2021-10-24,Sun,None,None,None,None,None,None,None,Home,WAS,Washington Football Team,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,7,None
202110280crd,202110280crd,October 28,2021-10-28,Thu,None,None,None,None,None,None,None,Away,CRD,Arizona Cardinals,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,8,None
202111070kan,202111070kan,November 7,2021-11-07,Sun,None,None,None,None,None,None,None,Away,KAN,Kansas City Chiefs,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,9,None
202111140gnb,202111140gnb,November 14,2021-11-14,Sun,None,None,None,None,None,None,None,Home,SEA,Seattle Seahawks,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,None,Reg,10,None


# Team

In [111]:
team = Team('GNB', 2021)
team

Green Bay Packers (GNB) - 2020

In [112]:
team.dataframe

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,offensive_simple_rating_system,pass_attempts,pass_completions,pass_first_downs,pass_net_yards_per_attempt,pass_touchdowns,pass_yards,penalties,percent_drives_with_points,percent_drives_with_turnovers,plays,points_against,points_contributed_by_offense,points_difference,points_for,post_season_result,rank,rush_attempts,rush_first_downs,rush_touchdowns,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play
GNB,GNB,1.8,358,28,6,16,5,3,8.8,Green Bay Packers,5.9,526,372,216,7.5,48,4106,84,49.7,5.6,990,369,290.75,140,509,Lost Conference Championship,1,443,114,16,2118,4.8,7.7,-1.1,11,0.813,13,6224,684,6.3


In [122]:
for i in range(2000, 2021):
    
    teams = Teams(2020).dataframes
        
    try:
        final_df = pd.concat([final_df, teams], axis=0)
        print('\n', f'Gots final_df already, appending year: {i}. SMOOCHES :*', '\n')
    except:
        final_df = teams.copy()
        print('\n', f'No final_df yet, creating from year: {i}', '\n')


 No final_df yet, creating from year: 2000 


 Gots final_df already, appending year: 2001. SMOOCHES :* 


 Gots final_df already, appending year: 2002. SMOOCHES :* 


 Gots final_df already, appending year: 2003. SMOOCHES :* 


 Gots final_df already, appending year: 2004. SMOOCHES :* 


 Gots final_df already, appending year: 2005. SMOOCHES :* 


 Gots final_df already, appending year: 2006. SMOOCHES :* 


 Gots final_df already, appending year: 2007. SMOOCHES :* 


 Gots final_df already, appending year: 2008. SMOOCHES :* 


 Gots final_df already, appending year: 2009. SMOOCHES :* 


 Gots final_df already, appending year: 2010. SMOOCHES :* 


 Gots final_df already, appending year: 2011. SMOOCHES :* 


 Gots final_df already, appending year: 2012. SMOOCHES :* 


 Gots final_df already, appending year: 2013. SMOOCHES :* 


 Gots final_df already, appending year: 2014. SMOOCHES :* 


 Gots final_df already, appending year: 2015. SMOOCHES :* 


 Gots final_df already, appending year

In [123]:
final_df

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,offensive_simple_rating_system,pass_attempts,pass_completions,pass_first_downs,pass_net_yards_per_attempt,pass_touchdowns,pass_yards,penalties,percent_drives_with_points,percent_drives_with_turnovers,plays,points_against,points_contributed_by_offense,points_difference,points_for,post_season_result,rank,rush_attempts,rush_first_downs,rush_touchdowns,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play
GNB,GNB,1.8,358,28,6,16,5,3,8.8,Green Bay Packers,5.9,526,372,216,7.5,48,4106,84,49.7,5.6,990,369,290.75,140,509,Lost Conference Championship,1,443,114,16,2118,4.8,7.7,-1.1,11,0.813,13,6224,684,6.3
BUF,BUF,0.6,397,38,11,16,11,3,7.9,Buffalo Bills,7.1,596,410,240,7.4,40,4620,102,49.4,11.8,1034,375,228.66,126,501,Lost Conference Championship,2,411,119,16,1723,4.2,7.7,-0.2,22,0.813,13,6343,941,6.1
TAM,TAM,2.8,364,44,5,16,12,5,8.6,Tampa Bay Buccaneers,6.5,626,410,238,7.1,42,4626,84,47.8,8.9,1017,355,246.24,137,492,Won Super Bowl,3,369,82,16,1519,4.1,9.4,0.8,17,0.688,11,6145,715,6.0
OTI,OTI,-3.1,381,36,5,16,7,5,3.3,Tennessee Titans,4.7,485,316,203,7.2,33,3653,86,47.9,7.2,1031,439,245.01,52,491,Lost WC,4,521,142,26,2690,5.2,1.6,-1.7,12,0.688,11,6343,783,6.2
NOR,NOR,4.5,367,21,9,16,8,4,9.1,New Orleans Saints,5.1,522,370,199,6.8,28,3758,98,45.5,9.0,1045,337,176.94,145,482,Lost Divisional,5,494,147,30,2265,4.6,9.6,0.5,17,0.750,12,6023,1005,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DEN,DEN,-1.7,308,38,9,16,23,11,-7.7,Denver Broncos,-4.3,556,317,176,5.9,21,3451,81,33.0,17.0,1030,446,-21.82,-123,323,None,28,442,94,13,1918,4.3,-6.0,1.7,32,0.313,5,5369,670,5.2
CIN,CIN,-2.1,318,25,13,16,11,11,-7.1,Cincinnati Bengals,-5.4,581,372,201,5.5,19,3448,83,33.0,12.8,1040,424,-5.13,-113,311,None,29,411,92,13,1668,4.1,-7.5,-0.4,24,0.281,4,5116,675,4.9
JAX,JAX,-5.3,310,25,9,16,16,15,-11.6,Jacksonville Jaguars,-6.5,616,387,205,5.6,25,3699,107,30.4,14.0,997,492,-6.12,-186,306,None,30,337,80,9,1519,4.5,-11.7,-0.1,25,0.063,1,5218,1071,5.2
NYG,NYG,2.3,297,28,11,16,11,10,-4.8,New York Giants,-6.7,517,321,178,5.3,12,3026,81,33.5,12.6,966,357,-26.79,-77,280,None,31,399,91,13,1768,4.4,-4.4,0.4,22,0.375,6,4794,634,5.0


In [115]:
teams = Teams(2020)
teams

Green Bay Packers (GNB)
Buffalo Bills (BUF)
Tampa Bay Buccaneers (TAM)
Tennessee Titans (OTI)
New Orleans Saints (NOR)
Kansas City Chiefs (KAN)
Baltimore Ravens (RAV)
Seattle Seahawks (SEA)
Indianapolis Colts (CLT)
Las Vegas Raiders (RAI)
Minnesota Vikings (MIN)
Pittsburgh Steelers (PIT)
Arizona Cardinals (CRD)
Cleveland Browns (CLE)
Miami Dolphins (MIA)
Atlanta Falcons (ATL)
Dallas Cowboys (DAL)
Houston Texans (HTX)
Los Angeles Chargers (SDG)
Detroit Lions (DET)
San Francisco 49ers (SFO)
Los Angeles Rams (RAM)
Chicago Bears (CHI)
Carolina Panthers (CAR)
Washington Football Team (WAS)
Philadelphia Eagles (PHI)
New England Patriots (NWE)
Denver Broncos (DEN)
Cincinnati Bengals (CIN)
Jacksonville Jaguars (JAX)
New York Giants (NYG)
New York Jets (NYJ)

In [116]:
teams.dataframes

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,offensive_simple_rating_system,pass_attempts,pass_completions,pass_first_downs,pass_net_yards_per_attempt,pass_touchdowns,pass_yards,penalties,percent_drives_with_points,percent_drives_with_turnovers,plays,points_against,points_contributed_by_offense,points_difference,points_for,post_season_result,rank,rush_attempts,rush_first_downs,rush_touchdowns,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play
GNB,GNB,1.8,358,28,6,16,5,3,8.8,Green Bay Packers,5.9,526,372,216,7.5,48,4106,84,49.7,5.6,990,369,290.75,140,509,Lost Conference Championship,1,443,114,16,2118,4.8,7.7,-1.1,11,0.813,13,6224,684,6.3
BUF,BUF,0.6,397,38,11,16,11,3,7.9,Buffalo Bills,7.1,596,410,240,7.4,40,4620,102,49.4,11.8,1034,375,228.66,126,501,Lost Conference Championship,2,411,119,16,1723,4.2,7.7,-0.2,22,0.813,13,6343,941,6.1
TAM,TAM,2.8,364,44,5,16,12,5,8.6,Tampa Bay Buccaneers,6.5,626,410,238,7.1,42,4626,84,47.8,8.9,1017,355,246.24,137,492,Won Super Bowl,3,369,82,16,1519,4.1,9.4,0.8,17,0.688,11,6145,715,6.0
OTI,OTI,-3.1,381,36,5,16,7,5,3.3,Tennessee Titans,4.7,485,316,203,7.2,33,3653,86,47.9,7.2,1031,439,245.01,52,491,Lost WC,4,521,142,26,2690,5.2,1.6,-1.7,12,0.688,11,6343,783,6.2
NOR,NOR,4.5,367,21,9,16,8,4,9.1,New Orleans Saints,5.1,522,370,199,6.8,28,3758,98,45.5,9.0,1045,337,176.94,145,482,Lost Divisional,5,494,147,30,2265,4.6,9.6,0.5,17,0.750,12,6023,1005,5.8
KAN,KAN,2.3,397,32,9,16,7,2,6.9,Kansas City Chiefs,4.5,630,420,255,7.4,40,4854,105,47.9,9.2,1057,362,280.82,111,473,Lost Super Bowl,6,403,110,13,1799,4.5,6.8,-0.1,16,0.875,14,6653,919,6.3
RAV,RAV,4.3,327,19,7,16,11,5,10.3,Baltimore Ravens,3.9,406,257,142,6.3,27,2739,104,45.3,10.6,993,303,148.30,165,468,Lost Divisional,7,555,166,24,3071,5.5,8.3,-2.0,18,0.688,11,5810,961,5.9
SEA,SEA,0.7,356,29,5,16,13,4,5.5,Seattle Seahawks,4.8,563,388,216,6.5,40,3941,84,44.4,10.1,1022,371,129.24,88,459,Lost WC,8,411,111,15,1971,4.8,5.5,0.0,18,0.750,12,5912,662,5.8
CLT,CLT,1.6,364,34,4,16,11,5,5.6,Indianapolis Colts,1.2,552,371,201,7.1,24,4053,94,44.4,8.8,1032,362,168.41,89,451,Lost WC,9,459,129,20,1996,4.3,2.8,-2.8,15,0.688,11,6049,899,5.9
RAI,RAI,-4.9,359,32,16,16,10,8,-2.8,Las Vegas Raiders,2.9,551,369,206,7.3,28,4217,98,47.9,14.8,1036,478,147.62,-44,434,None,10,457,121,20,1916,4.2,-2.0,0.8,26,0.500,8,6133,856,5.9
